# [Module 9] 로컬 앤드 포인트로 훈련된 모델 테스트 하기 


## 이 노트북은 아래와 같은 작업을 합니다.

- 1. 환경 셋업
- 2. 로컬 모드에서 배포 및 추론 테스트



---    
    

# 1. 환경 셋업

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

이전 노트북에서 인퍼런스 테스트를 완료한 티펙트를 가져옵니다.
- inference_docker_image: 사용자 정의 도커 이미지
- byom_artifact: 추론 코드를 포함한 모델 아티펙트

In [ ]:
import sagemaker
print("sm version: ", sagemaker.__version__)

## [중요] 모델 아티펙트 model.tar.gz 위치 복사
- 3.1.NCF-Training-Pipeline.ipynb 노트북에서 하단의 execution.list_steps() 의 결과에서 모델 아티펙트 (model.tar.gz) 의 경로를 복사하여 아래 byom_artifact 변수에 할당 합니다.
- 아래 그림 이미지 참조 하세요.

In [ ]:
byom_artifact = '<Enter S3 Model URL>'
# byom_artifact = 's3://sagemaker-us-east-1-XXXXXXX/ncf/repackage/model/2022-12-15-12-36-14/model.tar.gz'



![result_exe_step.png](img/result_exe_step.png)

In [ ]:
import sagemaker
import boto3

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [ ]:

inference_docker_image = f'763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-inference:1.8.1-gpu-py3'


# 2. 로컬 모드에서 배포 및 추론 테스트


## Python SDK를 통한 세이지 메이커 모델 생성
- 로컬 엔드포인트에 적재될 세이지 메이커 모델을 생성홥니다.
- 세이지 메이커 모델은 크게 아래 두가지 요소가 있습니다.
    - 인퍼런스 이미지 컨테이너 위치
    - 모델 아티펙트 위치

In [ ]:
from datetime import datetime
suffix = f"{datetime.today().strftime('%Y-%m-%d-%H-%M-%S')}"

# 로컬 노트북 인스턴스에서 사용할 로컬 세션
local_session = sagemaker.local.LocalSession()
local_session.config = {'local' : {'local_code':True}}
instance_type = 'local_gpu'
local_model_name = 'sm-local-model-' + str(suffix)
local_endpoint_name = 'local-endpoint-' + str(suffix)
print("local_endpoint_name: ", local_endpoint_name)

### 로컬 세이지 메이커 모델 정의

In [ ]:
from sagemaker.model import Model


def create_sm_model(ecr_image, model_artifact,model_name,  role, session):
    model = Model(
                name = model_name,
                image_uri = ecr_image,
                model_data = model_artifact,
                role=role,
                sagemaker_session= session,
                 )
    print("model: ", model)
    return model

local_model = create_sm_model(inference_docker_image, byom_artifact, 
                              local_model_name, role, 
                              local_session)    

### 로컬 엔드포인트 생성

- instance_type=='local' 시에는 최초 실행시에 인퍼런스 이미지를 다운로드로 하는 약 3분 걸리고, 이후에는 바로 실행이 됩니다.

In [ ]:
%%time

local_predictor = local_model.deploy(initial_instance_count=1, instance_type=instance_type,
                         endpoint_name = local_endpoint_name,                                   
                        )

## 테스트 데이터 세트 로딩
- 로컬에서 저장된 데이터를 가져와서 데이터를 변환 합니다.
- batch_size 만큼 데이터를 로딩하는 데이터 로더를 정의 합니다.

In [ ]:
import data_utils 
train_data, test_data, user_num ,item_num, train_mat = data_utils.load_all(test_num=100)

##  추론을 위한  데이터 세트 로딩
- 전부 데이터를 로딩할 필요가 없지만, 여기서는 기존에 사용한 함수를 이용하기 위해서 전체 데이터를 로드 합니다. 
    - 실제 데이터로 구현시에는 따로이 로드 함수를 사용하시기를 권장 합니다.


In [ ]:
class Params:
    def __init__(self):
        # self.epochs = 1        
        self.num_ng = 4
        self.batch_size = 256
        self.test_num_ng = 99
        self.factor_num = 32
        self.num_layers = 3
        self.dropout = 0.0
        # self.lr = 0.001
        self.top_k = 10
        self.out = True
        # self.gpu = "0"
                        
args = Params()
print("# of batch_size: ", args.batch_size)


import torch.utils.data as data

test_dataset = data_utils.NCFData(
		test_data, item_num, train_mat, 0, False)

test_loader = data.DataLoader(test_dataset,
		batch_size=args.test_num_ng+1, shuffle=False, num_workers=0)



## 추론할 Paylaod 하나를 생성

In [ ]:
for user, item, label in test_loader:   
    user_np = user.detach().cpu().numpy()
    item_np = item.detach().cpu().numpy()            
    break
payload = {'user':user_np.tolist(), 'item':item_np.tolist()}
print("payload: ", payload)


## 엔드 포인트 추론
- Boto3 invoke_endpoint() 로 추론

In [ ]:
import boto3

if instance_type == 'local_gpu':
    runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()    
    endpoint_name = local_endpoint_name
else:
    runtime_client = boto3.Session().client('sagemaker-runtime')



### [중요] JSON type 의 payload 를 String 으로 직렬화 해서 제공 함.
```python
payload_dump = json.dumps(payload)
```

In [ ]:
import json, time
from inference_utils import invoke_endpoint
payload_dump = json.dumps(payload)

start_time = time.time()
result = invoke_endpoint(runtime_client, endpoint_name, 
                         payload_dump,
                         content_type='application/json'
                        )

print("--- %s seconds ---" % (time.time() - start_time))
print('result: ', result)

## 로컬 엔드 포인트 삭제
- 기존에 생성한 세이지 메이커 모델, 앤드포인트 컨피그, 앤드포인트 삭제

In [ ]:
from inference_utils import delete_endpoint

client = sagemaker.local.LocalSagemakerClient()
delete_endpoint(client, endpoint_name)